In [19]:
import re
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from collections import Counter, defaultdict
from sqlalchemy import create_engine
from collections import defaultdict

In [20]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

In [21]:
df = pd.read_csv('tof_mri.csv')
df

,pt_no,measurement_procedure_id,procedure_occurrence_id,visit_occurrence_id,person_id,procedure_datetime,procedure_source_code,procedure_source_name,measurement_datetime,measurement_source_code,measurement_source_name,value_source_value
0,11705273,994,M0311705273201302130003,36525658,5434,2013-02-13 0:00,RM2121C,MRI Cardiac + CINE - Congenital heart disease ...,2013-04-26 15:55,NaN,NaN,"[Finding]\n2013-04-26 C.I> 심실비대, 부정맥 진행\r\ns..."
1,12170010,194,M0312170010200706270002,35164515,6372,2007-06-27 0:00,RM2121C,MRI Cardiac + CINE - Congenital heart disease ...,2007-08-29 16:26,NaN,NaN,[Finding]\n2007-08-29 \n\n\n[Conclusion]\ndi...
2,13865241,2366,M0313865241201609280004,36943990,13133,2016-09-28 0:00,RM2121C,MRI Cardiac + CINE - Congenital heart disease ...,2017-09-14 15:10,NaN,NaN,"[Finding]\n2017-09-14 C.I)\r\n# s/p TOF, MAP..."
3,15689816,4951,462394291,29527873,34235,2023-01-25 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2023-03-31 15:11,NaN,NaN,[Finding]\nCI:◈ TOF _- s/p (82-11-10)TOF total...
4,16404001,4082,300290820,19483297,43452,2021-01-29 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2021-07-16 15:07,NaN,NaN,[Finding]\n\n\n\n[Conclusion]\n2021-07-27 MR 판...
...,...,...,...,...,...,...,...,...,...,...,...,...
1674,79474102,3969,314544033,20395451,2966848,2021-04-08 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2021-04-22 15:07,NaN,NaN,[Finding]\nCI:PA wiith IVS_Ebstein anomaly\r\n...
1675,79474102,4102,338929706,21876470,2966848,2021-07-28 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2021-08-03 15:46,NaN,NaN,[Finding]\nCI:PA wiith IVS_Ebstein anomaly\r\n...
1676,79474102,5265,536416733,33562627,2966848,2023-12-04 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2023-12-04 15:57,NaN,NaN,[Finding]\nCI:PA with IVS_Ebstein anomaly\r\ns...
1677,79805636,4396,307743497,19945162,2990829,2021-03-08 0:00,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,2022-02-22 15:00,NaN,NaN,[Finding]\nCI:TOF 로 수술한 환자로 PR fraction 및 RV v...


In [22]:
df = df[['pt_no', 'measurement_datetime','procedure_source_code','procedure_source_name','value_source_value']].sort_values('measurement_datetime', ascending=True, ignore_index=True)
df

,pt_no,measurement_datetime,procedure_source_code,procedure_source_name,value_source_value
0,18727609,2004-10-19 16:23,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...
1,72490499,2004-11-04 15:27,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...
2,72073678,2004-11-25 15:10,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-11-25 \n\n\n[Conclusion]\nSe...
3,71567365,2004-12-15 15:55,RM2103,Cardiac MRI - Congenital heart disease,[Finding]\n2004-12-15 \n\n\n[Conclusion]\n1....
4,71912181,2004-12-17 15:49,RM2106,Cardiac MRI - General,[Finding]\n2004-12-17 1. PR\r\n2. regurgita...
...,...,...,...,...,...
1674,70098929,2023-12-18 16:20,RM21211,MRI Cardiac - Congenital (Cine+VENC) (noncontr...,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대..."
1675,18304190,2023-12-19 14:55,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,"[Finding]\nC.I. > S/P PVR, RVOT patch widening..."
1676,74663642,2023-12-26 14:38,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,[Finding]\nC.I. > ToF total repair (2006)\r\n\...
1677,72032413,2023-12-27 15:06,RM21211C,MRI Cardiac - Congenital (Cine+VENC+DE) (contr...,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...


In [23]:
df[['procedure_source_code','procedure_source_name']].value_counts()

procedure_source_code  procedure_source_name                                   
RM2121C                MRI Cardiac + CINE - Congenital heart disease (contrast)    1128
RM21211C               MRI Cardiac - Congenital (Cine+VENC+DE) (contrast)           460
RM21211                MRI Cardiac - Congenital (Cine+VENC) (noncontrast)            59
RM2131C                MRI Cardiac + CINE - General (contrast)                       14
RM2103                 Cardiac MRI - Congenital heart disease                         7
RM2111C                MRI Cardiac + CINE - Cardiomyopathy (contrast)                 6
RM2106                 Cardiac MRI - General                                          2
RM21111C               MRI Cardiac (Cine+DE+Mapping) (contrast)                       2
RM2131C                MRI Cardiac (Cine+DE) (contrast)                               1
Name: count, dtype: int64

In [24]:
text = df[df['procedure_source_code']=='RM2121C'].iloc[0, -1]
print(text)

[Finding]
2005-01-18   


[Conclusion]
1. severe PR
(regurgitation fraction= MPA reverse flow/MPA forward flow =54.26/112.03=0.48)
2. RVE but relatively conserved RV systolic function.
3. no TR, no PA stenosis.

                forward flow    reverse flow        net forward flow   (Normalized)
Aorta         73.19                 0                         73.19 (59.61)
MPA           112.03              54.26                  57.77 (47.06)
RPA           62.35                30.24                  32.11 (26.16)
LPA           39.29                16.95                   22.34 (18.19)


LV  EF      61.28%                           RV   EF    41.31%
       EDV   105.80ml (86.18)               EDV  239.83ml (195.35)  
       ESV   40.97ml (33.37)                     ESV   140.75ml (114.62)
       SV      64.83ml (52.81)                     SV     99.07ml (80.70)
       CO      5.89L/min (4.80)                  CO     9.01L/min (7.34)

BSA = 1.23m^2


In [25]:
text = df[df['procedure_source_code']=='RM2121C'].iloc[200, -1]
print(text)

[Finding]
2011-02-24   1. LV Function Measurement <absolute (normalized)>
 - Ejection Fraction : 55.3 %
 - End-Systolic Volume : 74.0 (51.8) ml
 - End-Diastolic Volume : 165.4 (115.8) ml
 - Stroke Volume : 91.4 (64.0) ml
 - Cardiac Output : 6.40 (4.48) L/min
 - BSA : 1.43 m2
 - Average Myocardial Mass : 96.2 (67.4) g
 - Heart Rate : 70 bpm

2. RV Function Measurement <absolute (normalized)>
 - Ejection Fraction : 42%
 - End-Systolic Volume : 102.1 (71.5) ml
 - End-Diastolic Volume :  175.9 (123.2) ml
 - Stroke Volume : 73.9 (51.7) ml
 - Cardiac Output : 5.17(3.62) L/min
 - BSA : 1.43 m2
 - Average Myocardial Mass : 0.0 (0.0) g
 - Heart Rate : 70 bpm


	Forward Vol.	Reverse Vol.	Net Forward Vol.		
Aorta	73.3	 0.016		73.29	
D aorta	51.32	 0		51.32	
SVC	26.64	 0		26.64	
IVC	46.23	 0		46.23	
dTA + SVC			51.32+26.64=77.96		
RPA	40.75	 1.44		39.31	
LPA	42.65	 9.8		32.86	
RPA+LPA	83.40	11.24		72.17	
MPA	92	 0.44		91.56	
TV1	65.52	 2.68		62.84	
TV2	74.41	 0.21		74.2


[Conclusion]
C.I.> TOF, S

In [26]:
import re
import pandas as pd
from tqdm import tqdm 

# 패턴 정의
patterns = {
    "Delayed Enhancement": re.compile(r"Delayed enhancement", re.DOTALL | re.IGNORECASE),
    "No Delayed Enhancement": re.compile(r"No.*?Delayed enhancement", re.IGNORECASE),
}

patterns_lv = {
    "LV EF": re.compile(r"LV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL),
    "LV EDV": re.compile(r"LV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV ESV": re.compile(r"LV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV SV": re.compile(r"LV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "LV CO": re.compile(r"LV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "LV MM": re.compile(r"LV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),
}

patterns_rv = {
    "RV EF": re.compile(r"RV.*?Ejection Fraction\s*:\s*([\d\.]+)\s*%", re.DOTALL | re.IGNORECASE),
    "RV EDV": re.compile(r"RV.*?End-Diastolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV ESV": re.compile(r"RV.*?End-Systolic Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV SV": re.compile(r"RV.*?Stroke Volume\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*ml", re.DOTALL | re.IGNORECASE),
    "RV CO": re.compile(r"RV.*?Cardiac Output\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*l/min", re.DOTALL | re.IGNORECASE),
    "RV MM": re.compile(r"RV.*?Average Myocardial Mass\s*:\s*([\d\.]+)\s*\(([\d\.]+)\)\s*g", re.DOTALL | re.IGNORECASE),
}

# 컬럼 정의
columns = ['pt_no', 'Date', 'Code', 'Text', 'Delayed Enhancement',
           'LV EF', 'LV EDV(Value)', 'LV EDV(Index)', 'LV ESV(Value)', 'LV ESV(Index)',
           'LV SV(Value)', 'LV SV(Index)', 'LV CO(Value)', 'LV CO(Index)', 'LV MM(Value)', 'LV MM(Index)',
           'RV EF', 'RV EDV(Value)', 'RV EDV(Index)', 'RV ESV(Value)', 'RV ESV(Index)',
           'RV SV(Value)', 'RV SV(Index)', 'RV CO(Value)', 'RV CO(Index)', 'RV MM(Value)', 'RV MM(Index)']

# 모든 패턴을 적용할 결과 리스트
all_results = []

# 각 행에 대해 패턴을 적용
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['value_source_value']
    results = {'pt_no': row['pt_no'], 'Date': row['measurement_datetime'], 'Code': row['procedure_source_code'], 'Text':text}

    # 기본 패턴
    for key, pattern in patterns.items():
        if (key in results) or (key+'(Value)' in results) or (key+'(Index)' in results):
            continue
        matches = pattern.findall(text)
        for match in matches:
            if 'Delayed Enhancement' in key:
                results[key] = "Yes"
                if 'No Delayed Enhancement' in key:
                    results['Delayed Enhancement'] = "No"

    # LV 패턴
    for key, pattern in patterns_lv.items():
        if (key in results) or (key+'(Value)' in results) or (key+'(Index)' in results):
            continue
        matches = pattern.findall(text)
        for match in matches:
            if 'EF' in key:
                results[key] = match
            else:
                results[key+'(Value)'] = match[0]
                results[key+'(Index)'] = match[1]

    # RV 패턴
    for key, pattern in patterns_rv.items():
        if (key in results) or (key+'(Value)' in results) or (key+'(Index)' in results):
            continue
        matches = pattern.findall(text)
        for match in matches:
            if 'EF' in key:
                results[key] = match
            else:
                results[key+'(Value)'] = match[0]
                results[key+'(Index)'] = match[1]

    all_results.append(results)

# DataFrame 생성
all_results_df = pd.DataFrame(all_results)
all_results_df = all_results_df[columns]
all_results_df


  0%|          | 0/1679 [00:00<?, ?it/s]

100%|██████████| 1679/1679 [00:01<00:00, 1641.76it/s]


,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,18727609,2004-10-19 16:23,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72490499,2004-11-04 15:27,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72073678,2004-11-25 15:10,RM2103,[Finding]\n2004-11-25 \n\n\n[Conclusion]\nSe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71567365,2004-12-15 15:55,RM2103,[Finding]\n2004-12-15 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71912181,2004-12-17 15:49,RM2106,[Finding]\n2004-12-17 1. PR\r\n2. regurgita...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,70098929,2023-12-18 16:20,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",NaN,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
1675,18304190,2023-12-19 14:55,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",NaN,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
1676,74663642,2023-12-26 14:38,RM21211C,[Finding]\nC.I. > ToF total repair (2006)\r\n\...,Yes,53.2,112.4,66.1,52.6,30.9,59.8,35.2,5.68,3.34,86.4,50.8,41.7,259.2,150.4,151.0,87.6,108.2,62.8,10.28,5.97,NaN,NaN
1677,72032413,2023-12-27 15:06,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,NaN,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3


In [27]:
# Create a DataFrame from the extracted results
all_results_df = pd.DataFrame(all_results)

# Check which columns exist in the dataframe
existing_columns = [col for col in columns if col in all_results_df.columns]

# Select only the columns that exist in the dataframe
all_results_df = all_results_df[existing_columns]

In [28]:
all_results_df[all_results_df['RV EF'].notna()]

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
50,70336777,2006-01-12 16:00,RM2121C,[Finding]\n2006-01-12 \n\n\n[Conclusion]\n1....,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,12170010,2007-07-04 16:19,RM2121C,[Finding]\n2007-07-04 \n\n\n[Conclusion]\n1....,Yes,40.2,188.7,114.1,112.9,68.3,75.8,45.8,4.70,2.84,179.8,108.8,20.8,119.4,72.2,94.5,57.2,24.9,15.0,1.54,1.54,97.3,58.9
107,36235735,2007-07-05 16:28,RM2131C,[Finding]\n2007-07-05 \n\n\n[Conclusion]\n1....,NaN,53.7,137.5,88.0,63.7,40.7,73.9,47.2,5.17,3.31,55.8,35.7,47.0,196.4,125.6,104.1,66.6,92.3,59.0,6.46,4.13,55.8,35.7
108,73251039,2007-07-26 16:36,RM2121C,[Finding]\n2007-07-26 \n\n\n[Conclusion]\n1....,NaN,63.5,63.6,75.6,23.2,27.6,40.4,48.0,2.99,3.56,29.4,35.0,38.2,48.6,57.8,30.1,35.8,18.6,22.1,1.37,1.63,29.4,35.0
109,36482531,2007-08-08 14:08,RM2121C,[Finding]\n2007-08-08 CI :\r\ns/p TOF total ...,NaN,51.7,225.9,165.0,109.2,79.8,116.7,85.2,6.65,4.86,74.9,54.7,60.2,171.8,125.5,68.4,50.0,103.4,75.5,5.89,4.30,74.9,54.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,70098929,2023-12-18 16:20,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",NaN,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
1675,18304190,2023-12-19 14:55,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",NaN,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
1676,74663642,2023-12-26 14:38,RM21211C,[Finding]\nC.I. > ToF total repair (2006)\r\n\...,Yes,53.2,112.4,66.1,52.6,30.9,59.8,35.2,5.68,3.34,86.4,50.8,41.7,259.2,150.4,151.0,87.6,108.2,62.8,10.28,5.97,NaN,NaN
1677,72032413,2023-12-27 15:06,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,NaN,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3


In [29]:
all_results_df[all_results_df['Delayed Enhancement']=='No']

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
118,15469182,2008-01-29 16:09,RM2121C,[Finding]\n2008-01-29 \n\n\n[Conclusion]\nC....,No,57.3,307.7,161.9,131.5,69.2,176.2,92.7,10.75,5.66,173.8,91.4,36.2,329.2,173.2,209.9,110.4,119.3,62.8,7.28,3.83,173.8,91.4
239,70377831,2011-06-24 16:31,RM2121C,[Finding]\n2011-06-24 C.I.>s/p SA + SV (RV ...,No,56.3,197.1,134.3,86.2,58.7,110.9,75.6,8.68,5.9,0.0,0.0,56.3,197.1,134.3,86.2,58.7,110.9,75.6,8.68,5.9,0.0,0.0
269,71167020,2011-10-17 15:08,RM2121C,[Finding]\n2011-10-17 1. LV Function Measure...,No,55.9,110.6,73.3,48.8,32.4,61.8,41.0,3.71,2.46,70.3,46.6,48.0,201.3,133.4,104.7,69.4,96.6,64.0,5.80,3.84,0.0,0.0
274,72473728,2011-11-01 15:47,RM2121C,[Finding]\n2011-11-01 1. LV Function Measure...,No,43.4,146.8,86.3,83.1,48.9,63.7,37.4,5.67,3.33,99.6,58.6,25.7,252.5,148.4,187.7,110.4,64.8,38.1,5.77,3.39,0.0,0.0
296,73152954,2012-01-26 15:33,RM2121C,[Finding]\n2012-01-26 C.I> s/p PA with IVS로 ...,No,47.0,148.4,108.4,78.7,57.5,69.7,50.9,5.79,4.22,78.4,57.2,45.1,223.7,163.4,122.7,89.6,101.0,73.8,8.38,6.12,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1617,15162520,2023-07-10 14:35,RM21211C,[Finding]\nC.I.>#TOF 수술 후 PR fraction 및 RV vol...,No,53.5,138.8,81.6,64.6,38.0,74.2,43.6,6.38,3.75,63.8,37.5,46.0,242.7,142.8,131.1,77.1,111.6,65.6,9.60,5.65,63.8,37.5
1640,72384866,2023-09-15 14:22,RM21211C,[Finding]\nCI: # TOF.\r\n- s/p Total correctio...,No,56.9,159.5,113.9,68.7,49.1,90.8,64.9,4.63,3.31,73.8,52.7,56.9,155.7,111.2,70.0,50.0,85.7,61.2,4.37,3.12,NaN,NaN
1670,77550776,2023-11-24 15:20,RM21211C,[Finding]\n# PA with IVS \r\n s/p 2014/7/31 pu...,No,62.9,63.9,63.9,23.7,23.7,40.2,40.2,3.46,3.46,31.7,31.7,60.3,59.9,59.9,23.8,23.8,36.1,36.1,3.10,3.10,NaN,NaN
1671,79474102,2023-12-04 15:57,RM21211C,[Finding]\nCI:PA with IVS_Ebstein anomaly\r\ns...,No,55.8,55.4,79.1,24.5,35.0,30.9,44.1,2.97,4.24,81.6,116.6,65.0,31.1,44.4,10.9,15.6,20.2,28.9,1.94,2.77,81.6,116.6


In [30]:
all_results_df['Delayed Enhancement'] = all_results_df['Delayed Enhancement'].fillna('No')
all_results_df[all_results_df['Delayed Enhancement']=='No']

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,18727609,2004-10-19 16:23,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72490499,2004-11-04 15:27,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72073678,2004-11-25 15:10,RM2103,[Finding]\n2004-11-25 \n\n\n[Conclusion]\nSe...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71567365,2004-12-15 15:55,RM2103,[Finding]\n2004-12-15 \n\n\n[Conclusion]\n1....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71912181,2004-12-17 15:49,RM2106,[Finding]\n2004-12-17 1. PR\r\n2. regurgita...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1671,79474102,2023-12-04 15:57,RM21211C,[Finding]\nCI:PA with IVS_Ebstein anomaly\r\ns...,No,55.8,55.4,79.1,24.5,35.0,30.9,44.1,2.97,4.24,81.6,116.6,65.0,31.1,44.4,10.9,15.6,20.2,28.9,1.94,2.77,81.6,116.6
1673,79179421,2023-12-12 14:51,RM21211C,[Finding]\nCI: f/u\r\n# PA IVS\r\n - s/p RV ov...,No,60.0,65.7,93.9,26.3,37.6,39.4,56.3,2.56,3.66,31.3,44.7,51.2,115.4,164.9,56.3,80.4,59.1,84.4,3.84,5.49,31.3,44.7
1674,70098929,2023-12-18 16:20,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",No,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
1675,18304190,2023-12-19 14:55,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",No,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7


In [31]:
all_results_df.dropna()

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
106,12170010,2007-07-04 16:19,RM2121C,[Finding]\n2007-07-04 \n\n\n[Conclusion]\n1....,Yes,40.2,188.7,114.1,112.9,68.3,75.8,45.8,4.70,2.84,179.8,108.8,20.8,119.4,72.2,94.5,57.2,24.9,15.0,1.54,1.54,97.3,58.9
107,36235735,2007-07-05 16:28,RM2131C,[Finding]\n2007-07-05 \n\n\n[Conclusion]\n1....,No,53.7,137.5,88.0,63.7,40.7,73.9,47.2,5.17,3.31,55.8,35.7,47.0,196.4,125.6,104.1,66.6,92.3,59.0,6.46,4.13,55.8,35.7
108,73251039,2007-07-26 16:36,RM2121C,[Finding]\n2007-07-26 \n\n\n[Conclusion]\n1....,No,63.5,63.6,75.6,23.2,27.6,40.4,48.0,2.99,3.56,29.4,35.0,38.2,48.6,57.8,30.1,35.8,18.6,22.1,1.37,1.63,29.4,35.0
109,36482531,2007-08-08 14:08,RM2121C,[Finding]\n2007-08-08 CI :\r\ns/p TOF total ...,No,51.7,225.9,165.0,109.2,79.8,116.7,85.2,6.65,4.86,74.9,54.7,60.2,171.8,125.5,68.4,50.0,103.4,75.5,5.89,4.30,74.9,54.7
110,71631583,2007-08-09 15:35,RM2121C,[Finding]\n2007-08-09 1. LV Function Measure...,No,59.7,145.5,85.8,58.6,34.6,86.9,51.3,5.13,3.02,93.7,55.3,34.9,170.0,100.3,110.7,65.3,59.3,35.0,3.50,2.06,93.7,55.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,79179421,2023-12-12 14:51,RM21211C,[Finding]\nCI: f/u\r\n# PA IVS\r\n - s/p RV ov...,No,60.0,65.7,93.9,26.3,37.6,39.4,56.3,2.56,3.66,31.3,44.7,51.2,115.4,164.9,56.3,80.4,59.1,84.4,3.84,5.49,31.3,44.7
1674,70098929,2023-12-18 16:20,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",No,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
1675,18304190,2023-12-19 14:55,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",No,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
1677,72032413,2023-12-27 15:06,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,No,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3


In [32]:
all_results_df[all_results_df['RV EF'].notna()]

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
50,70336777,2006-01-12 16:00,RM2121C,[Finding]\n2006-01-12 \n\n\n[Conclusion]\n1....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,12170010,2007-07-04 16:19,RM2121C,[Finding]\n2007-07-04 \n\n\n[Conclusion]\n1....,Yes,40.2,188.7,114.1,112.9,68.3,75.8,45.8,4.70,2.84,179.8,108.8,20.8,119.4,72.2,94.5,57.2,24.9,15.0,1.54,1.54,97.3,58.9
107,36235735,2007-07-05 16:28,RM2131C,[Finding]\n2007-07-05 \n\n\n[Conclusion]\n1....,No,53.7,137.5,88.0,63.7,40.7,73.9,47.2,5.17,3.31,55.8,35.7,47.0,196.4,125.6,104.1,66.6,92.3,59.0,6.46,4.13,55.8,35.7
108,73251039,2007-07-26 16:36,RM2121C,[Finding]\n2007-07-26 \n\n\n[Conclusion]\n1....,No,63.5,63.6,75.6,23.2,27.6,40.4,48.0,2.99,3.56,29.4,35.0,38.2,48.6,57.8,30.1,35.8,18.6,22.1,1.37,1.63,29.4,35.0
109,36482531,2007-08-08 14:08,RM2121C,[Finding]\n2007-08-08 CI :\r\ns/p TOF total ...,No,51.7,225.9,165.0,109.2,79.8,116.7,85.2,6.65,4.86,74.9,54.7,60.2,171.8,125.5,68.4,50.0,103.4,75.5,5.89,4.30,74.9,54.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1674,70098929,2023-12-18 16:20,RM21211,"[Finding]\nC.I. > RV volume 및 RVEF, PR 정도 등에 대...",No,51.3,116.1,75.9,56.6,37.0,59.5,38.9,4.40,2.88,74.5,48.7,43.4,167.9,109.4,95.0,61.9,72.9,47.5,5.40,3.52,74.5,48.7
1675,18304190,2023-12-19 14:55,RM21211C,"[Finding]\nC.I. > S/P PVR, RVOT patch widening...",No,71.8,150.4,71.6,42.4,20.2,108.0,51.4,8.21,3.91,108.5,51.7,38.4,253.6,120.8,156.1,74.3,97.5,46.4,7.41,3.53,108.5,51.7
1676,74663642,2023-12-26 14:38,RM21211C,[Finding]\nC.I. > ToF total repair (2006)\r\n\...,Yes,53.2,112.4,66.1,52.6,30.9,59.8,35.2,5.68,3.34,86.4,50.8,41.7,259.2,150.4,151.0,87.6,108.2,62.8,10.28,5.97,NaN,NaN
1677,72032413,2023-12-27 15:06,RM21211C,[Finding]\nC.I. > TOF 후 PPVR 대상자입니다. MAIN PA 가...,No,63.5,119.4,66.3,43.6,24.2,75.8,42.1,9.02,5.01,96.0,53.3,30.3,243.5,135.3,169.7,94.3,73.8,41.0,8.78,4.88,96.0,53.3


In [33]:
all_results_df[all_results_df['RV EF'].notna()].to_csv('MRI_Regex.csv', index=False, encoding='utf-8-sig')

In [34]:
# 아무런 항목이 추출되지 않은 경우 확인

columns_to_check = [
    'LV EF', 'LV EDV(Value)',
    'LV EDV(Index)', 'LV SV(Value)', 'LV SV(Index)', 'LV CO(Value)',
    'LV CO(Index)', 'LV MM(Value)', 'LV MM(Index)', 'RV EF',
    'RV EDV(Value)', 'RV EDV(Index)', 'RV SV(Value)', 'RV SV(Index)',
    'RV CO(Value)', 'RV CO(Index)', 'RV MM(Value)', 'RV MM(Index)'
]

all_results_df[all_results_df[columns_to_check].isnull().all(axis=1)]

,pt_no,Date,Code,Text,Delayed Enhancement,LV EF,LV EDV(Value),LV EDV(Index),LV ESV(Value),LV ESV(Index),LV SV(Value),LV SV(Index),LV CO(Value),LV CO(Index),LV MM(Value),LV MM(Index),RV EF,RV EDV(Value),RV EDV(Index),RV ESV(Value),RV ESV(Index),RV SV(Value),RV SV(Index),RV CO(Value),RV CO(Index),RV MM(Value),RV MM(Index)
0,18727609,2004-10-19 16:23,RM2103,[Finding]\n2004-10-19 [추가판독]\r\n[Finding]\r\...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,72490499,2004-11-04 15:27,RM2103,[Finding]\n2004-11-04 CARDIAC MRI - CONGENIT...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,72073678,2004-11-25 15:10,RM2103,[Finding]\n2004-11-25 \n\n\n[Conclusion]\nSe...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71567365,2004-12-15 15:55,RM2103,[Finding]\n2004-12-15 \n\n\n[Conclusion]\n1....,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,71912181,2004-12-17 15:49,RM2106,[Finding]\n2004-12-17 1. PR\r\n2. regurgita...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,73534152,2019-08-20 16:39,RM2131C,"[Finding]\n2019-08-20 # TOF, PR, LPA stenosi...",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1291,16404001,2021-07-16 15:07,RM21211C,[Finding]\n\n\n\n[Conclusion]\n2021-07-27 MR 판...,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1297,73534152,2021-08-02 14:59,RM21211,"[Finding]\n# TOF, PR, LPA stenosis \r\n - s/p ...",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,74995457,2022-04-20 14:45,RM21211C,"[Finding]\nPulmonary atresia(valvar), VSD, LPA...",No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
ddf = all_results_df[all_results_df['RV EF'].notna()]
unique_pt_no_count = ddf['pt_no'].nunique()
print(unique_pt_no_count)

745
